In [ ]:
# === Imports === #
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import yaml
import os
from google_sheet_api import GoogleSheetsUploader

# === Setup Paths === #
BASE_DIR = os.getcwd()
CONFIG_PATH = os.path.join(BASE_DIR, "config.yaml")
CREDENTIAL_PATH = os.path.join(BASE_DIR, "credential_google_sheets.json")

# === Load Configuration === #
def load_config(path):
    """Load configuration from YAML file."""
    with open(path, "r") as file:
        return yaml.safe_load(file)

config = load_config(CONFIG_PATH)

# === Initialize MT5 Connection === #
if not mt5.initialize():
    print("MT5 initialization failed.")
    quit()

# === Retrieve Account Information === #
account_info = mt5.account_info()
if account_info is None:
    print("Failed to retrieve account information.")
    mt5.shutdown()
    quit()

# === Extract Balance, Equity, and Floating Loss === #
balance = account_info.balance
equity = account_info.equity
floating_loss = account_info.profit

# === Display Results === #
print("=== Account Information ===")
print(f"Balance: {balance:.2f} USD")
print(f"Equity: {equity:.2f} USD")
print(f"Floating Loss: {floating_loss:.2f} USD")

# === Fetch and Process Running Trades === #
fetcher = GoogleSheetsUploader(CREDENTIAL_PATH, "Financial Report - Indonesia")
df = fetcher.get_sheet_as_dataframe("Forex")
positions = mt5.positions_get()
df_positions = pd.DataFrame([pos._asdict() for pos in positions]) if positions else pd.DataFrame()

if df_positions.empty:
    print("No running trades found. Error:", mt5.last_error())
else:
    # Filter and rename columns
    df_positions = df_positions[["type", "volume", "price_open", "price_current", "sl", "tp", "profit", "symbol"]]
    df_positions["type"] = df_positions["type"].replace({0: "Buy", 1: "Sell"})
    df_positions.rename(columns={
        "symbol": "Symbol",
        "type": "Action",
        "volume": "Lot",
        "price_open": "Price",
        "price_current": "Price Current",
        "sl": "SL",
        "tp": "TP",
        "profit": "Profit"
    }, inplace=True)

    # Aggregate by Symbol and Action
    def aggregate_group(group):
        symbol = group["Symbol"].iloc[0]
        action = group["Action"].iloc[0]
        lot = group["Lot"].sum()
        warnings = []

        # === Lot mismatch check === #
        df_lot_value = df.loc[(df.iloc[:, 0] == symbol) & (df.iloc[:, 2] == action), df.columns[5]].sum()
        if not pd.isna(df_lot_value) and round(df_lot_value, 2) > round(lot, 2):
            warnings.append(f"Lot Mismatch: Expected {round(df_lot_value, 2)}, Found {round(lot, 2)}")

        # === SL/TP mismatch check === #
        df_sl_value = df.loc[(df.iloc[:, 0] == symbol) & (df.iloc[:, 2] == action), df.columns[7]].sum()
        sl_mode = group["SL"].mode().iloc[0] if not group["SL"].mode().empty else np.nan
        if group["SL"].nunique() > 1 or (df_sl_value != 0 and round(df_sl_value, 2) != round(sl_mode, 2)):
            warnings.append("SL Mismatch")

        df_tp_value = df.loc[(df.iloc[:, 0] == symbol) & (df.iloc[:, 2] == action), df.columns[8]].sum()
        tp_mode = group["TP"].mode().iloc[0] if not group["TP"].mode().empty else np.nan
        if group["TP"].nunique() > 1 or round(df_tp_value, 2) != round(tp_mode, 2):
            warnings.append("TP Mismatch")

        # Combine warnings
        warning_message = "; ".join(warnings) if warnings else np.nan

        return pd.Series({
            "Lot": lot,
            "Price": (group["Price"] * group["Lot"]).sum() / group["Lot"].sum(),
            "Price Current": group["Price Current"].mean(),
            "SL": group["SL"].mode().iloc[0] if not group["SL"].empty else np.nan,
            "TP": group["TP"].mode().iloc[0] if not group["TP"].empty else np.nan,
            "Profit": group["Profit"].sum(),
            "Warning": warning_message
        })

    df_positions = (
        df_positions.groupby(["Symbol", "Action"], as_index=False)
        .apply(lambda group: aggregate_group(group))
        .reset_index(drop=True)
    )

# === Shutdown MT5 Connection === #
mt5.shutdown()

# === Calculate Coefficients === #
df_positions["Coeff"] = np.abs(df_positions["Profit"] / (df_positions["Price Current"] - df_positions["Price"])) / (df_positions["Lot"] * 100)
df_positions["Pips"] = np.abs(df_positions["TP"] - df_positions["SL"])

# Merging the Type column based on Symbol
df_type = df[[df.columns[0], df.columns[1]]].drop_duplicates()
df_type.columns = ["Symbol", "Type"]
df_positions = pd.merge(df_positions, df_type, on="Symbol", how="left")

# Handling missing types (optional)
df_positions['Type'].fillna("Unknown", inplace=True)

df_positions

# === Update Coefficients in Config (Exclude NaN) === #
df_avg_coeff = df_positions.groupby("Symbol")["Coeff"].mean().reset_index()
differences = {}

for _, row in df_avg_coeff.iterrows():
    symbol = row["Symbol"]
    avg_coeff = round(row["Coeff"], 5)
    
    # Only update if coefficient is not NaN
    if not pd.isna(avg_coeff):
        if symbol in config.get("symbol_coefficients", {}):
            old_coeff = config["symbol_coefficients"].get(symbol, avg_coeff)
            if old_coeff != avg_coeff:
                differences[symbol] = {"Old": old_coeff, "New": avg_coeff}
        
        # Add or update the coefficient
        config["symbol_coefficients"][symbol] = avg_coeff

# === Save Updated Configuration === #
with open(CONFIG_PATH, "w") as file:
    yaml.safe_dump(config, file)

# === Display Changed Coefficients === #
print("Changed symbol_coefficients:")
for symbol, change in differences.items():
    print(f"{symbol}: {change['Old']} -> {change['New']}")

# Creating new rows for Balance, Equity, and Floating Loss
df_balance = pd.DataFrame([
    {"Symbol": "Balance", "Price": balance},
    {"Symbol": "Equity", "Price": equity},
    {"Symbol": "Floating Loss", "Price": floating_loss}
])

# Inserting the new rows at the top of the DataFrame
df_final = pd.concat([df_balance, df_positions], ignore_index=True)

# === Uploade Updated Dataframe === #
uploader = GoogleSheetsUploader(CREDENTIAL_PATH, "Financial Report - Indonesia")
uploader.upload_dataframe(df_final, "Forex Summary", replace=True)

# === Display Final Merged DataFrame === #
print("Final Merged DataFrame:")
df_final

=== Account Information ===
Balance: 4534.15 USD
Equity: 4064.68 USD
Floating Loss: -1718.35 USD
✅ Successfully retrieved data from 'Forex' as a DataFrame.


C:\Users\syahr\AppData\Local\Temp\ipykernel_2552\2936804534.py:106: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: aggregate_group(group))
C:\Users\syahr\AppData\Local\Temp\ipykernel_2552\2936804534.py:123: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation 

Changed symbol_coefficients:
AAVEUSD: 0.5003 -> 0.50029
AIR.EPA: 1.12322 -> 1.1193
ALGOUSD: 25.00731 -> 24.99296
APTUSD: 2.99977 -> 3.0009
AUDCAD: 716.92606 -> 716.97234
AUDCHF: 1198.95833 -> 1198.55222
AUDJPY: 6.89162 -> 6.89053
AUDNZD: 590.65689 -> 590.98565
AVAXUSD: 1.00036 -> 1.00034
AZN.LSE: 1.33284 -> 1.33333
BATS.LSE: 1.33189 -> 1.33164
BBVA.BM: 1.12341 -> 1.12545
BCHUSD: 0.10017 -> 0.09993
BEI.XE: 1.00084 -> 1.00137
BMW.XE: 1.00063 -> 1.00067
BNBUSD: 0.10027 -> 0.1
BNP.EPA: 1.08108 -> 1.14458
CADCHF: 1199.17012 -> 1198.38057
CADJPY: 6.88828 -> 6.8958
CHFJPY: 6.88738 -> 6.89158
DKI.TSE: 0.00689 -> 0.0069
ENI.MIL: 1.12025 -> 1.12121
EURAUD: 642.60043 -> 642.41239
EURCAD: 716.94473 -> 716.8608
EURCHF: 1199.57537 -> 1199.57983
EURGBP: 1332.92782 -> 1332.34955
EURJPY: 6.89365 -> 6.89416
EURMXN: 51.23714 -> 51.25267
EURNZD: 590.87681 -> 591.08139
EURZAR: 55.42763 -> 55.46321
FETUSD: 29.99593 -> 29.99607
FILUSD: 0.99983 -> 0.99984
GBPAUD: 642.62766 -> 642.30161
GBPCAD: 716.36946 -> 71

In [ ]:
import time
import MetaTrader5 as mt5
import pandas as pd
import os
from google_sheet_api import GoogleSheetsUploader

# === Setup Paths === #
BASE_DIR = os.getcwd()
CONFIG_PATH = os.path.join(BASE_DIR, "config.yaml")
CREDENTIAL_PATH = os.path.join(BASE_DIR, "credential_google_sheets.json")

# === Initialize MT5 Connection === #
if not mt5.initialize():
    print("MT5 initialization failed.")
    quit()

fetcher = GoogleSheetsUploader(CREDENTIAL_PATH, "Financial Report - Indonesia")

# Load DataFrame from Google Sheets
df = fetcher.get_sheet_as_dataframe("Forex")
df.columns = df.iloc[6]
df = df.iloc[7:-3].reset_index(drop=True)

# Initialize counters for success and failure
success_count = 0
failure_count = 0

# Fetch unique symbols from running trades in MT5
positions = mt5.positions_get()
if positions is None or len(positions) == 0:
    print("No running trades found. Exiting.")
    mt5.shutdown()
    quit()

# Get unique symbols from MT5 positions
symbol_list = sorted(list(set(pos.symbol for pos in positions)))

# Convert positions to DataFrame for easy filtering
df_positions = pd.DataFrame([pos._asdict() for pos in positions])

# Iterate through each unique symbol from MT5 positions
for symbol in symbol_list:
    print(f"\n=== Checking Symbol: {symbol} ===")
    for action in ["Buy", "Sell"]:
        print(f"  Action: {action}")
        action_type = 0 if action == "Buy" else 1
        filtered_positions = df_positions[(df_positions['symbol'] == symbol) & (df_positions['type'] == action_type)]
        
        if filtered_positions.empty:
            print(f"    ⚠️ No {action} positions running. Skipping.")
            continue

        # Filter rows for the current symbol and action from Google Sheets
        df_filtered = df[(df["Symbol"] == symbol) & (df["Action"] == action)]
        
        if df_filtered.empty:
            print(f"    ⚠️ No matching TP/SL settings in Google Sheets.")
            continue
        
        tp_value = round(df_filtered['TP'].values[0], 5)
        sl_value = round(df_filtered['SL'].values[0], 5)

        # Iterate over filtered positions for modification
        for _, pos in filtered_positions.iterrows():
            order_id = pos['ticket']
            current_tp = pos['tp']
            current_sl = pos['sl']

            # Check for differences and print
            print(f"    Position ID: {order_id}")
            print(f"    Current TP: {current_tp}, Current SL: {current_sl}")
            print(f"    Target TP: {tp_value}, Target SL: {sl_value}")

            if current_tp == tp_value and current_sl == sl_value:
                print(f"    ✅ No changes needed for {action} position {order_id}.")
                continue
            else:
                print(f"    ⚠️ TP/SL difference detected. Attempting to update...")

            # Modify TP/SL using position ID
            request = {
                "action": mt5.TRADE_ACTION_SLTP,
                "position": order_id,
                "sl": sl_value,
                "tp": tp_value
            }

            # Send modification request
            result = mt5.order_send(request)

            # Check if result is None (Error)
            if result is None:
                print(f"    ⚠️ Failed to modify {action} position {order_id}. Error: Request returned None.")
                failure_count += 1
                continue

            # Check if the modification was successful
            if result.retcode == mt5.TRADE_RETCODE_DONE:
                print(f"    ✅ {action} position {order_id} modified successfully.")
                print(f"    Before: TP={current_tp}, SL={current_sl}")
                print(f"    After:  TP={tp_value}, SL={sl_value}\n")
                success_count += 1
            else:
                print(f"    ⚠️ Failed to modify {action} position {order_id}. Error: {result.comment}")
                failure_count += 1

# === Shutdown MT5 Connection === #
mt5.shutdown()

# === Summary Report ===
print("\n=== TP/SL Modification Summary ===")
print(f"Total Successful Modifications: {success_count}")
print(f"Total Failed Modifications: {failure_count}")

✅ Successfully retrieved data from 'Forex' as a DataFrame.

=== Checking Symbol: AAPL.NAS ===
  Action: Buy
    Position ID: 3297549570
    Current TP: 217.47, Current SL: 67.34
    Target TP: 217.94, Target SL: 67.35
    ⚠️ TP/SL difference detected. Attempting to update...
    ⚠️ Failed to modify Buy position 3297549570. Error: Market closed
  Action: Sell
    ⚠️ No Sell positions running. Skipping.

=== Checking Symbol: AAVEUSD ===
  Action: Buy
    Position ID: 3302602638
    Current TP: 239.76, Current SL: 0.01
    Target TP: 239, Target SL: 0.01
    ⚠️ TP/SL difference detected. Attempting to update...
    ⚠️ Failed to modify Buy position 3302602638. Error: Request returned None.
  Action: Sell
    ⚠️ No Sell positions running. Skipping.

=== Checking Symbol: ABBV.NYSE ===
  Action: Buy
    Position ID: 3298446412
    Current TP: 200.55, Current SL: 125.12
    Target TP: 200.25, Target SL: 125.24
    ⚠️ TP/SL difference detected. Attempting to update...
    ⚠️ Failed to modify Bu